In [7]:
#dependencies
import pandas as pd
from sportsreference.nba.teams import Teams
from scipy.stats import pearsonr

In [8]:
#pulling NBA team data from 2017-2019 for model training, adding year to each dataframe, and concatenating all 3 
NBA17 = Teams("2017").dataframes
NBA17 = NBA17.assign(Season=['2017'] * 30)
NBA18 = Teams("2018").dataframes
NBA18= NBA18.assign(Season=['2018'] * 30)
NBA19 = Teams("2019").dataframes
NBA19 = NBA19.assign(Season=['2019'] * 30)
NBA20 = Teams("2020").dataframes
NBA20 = NBA19.assign(Season=['2020'] * 30)
NBA = [NBA17, NBA18, NBA19, NBA20]
NBA = pd.concat(NBA)
#changing datatype of Season to match subsequent standings dataframe for future merge
NBA['Season'] = NBA['Season'].astype('int64')
NBA

,abbreviation,assists,blocks,defensive_rebounds,field_goal_attempts,field_goal_percentage,field_goals,free_throw_attempts,free_throw_percentage,free_throws,...,steals,three_point_field_goal_attempts,three_point_field_goal_percentage,three_point_field_goals,total_rebounds,turnovers,two_point_field_goal_attempts,two_point_field_goal_percentage,two_point_field_goals,Season
GSW,GSW,2491,555,2873,7140,0.495,3532,1850,0.788,1457,...,785,2562,0.383,982,3643,1211,4578,0.557,2550,2017
HOU,HOU,2070,352,2746,7152,0.462,3305,2177,0.766,1667,...,670,3306,0.357,1181,3638,1240,3846,0.552,2124,2017
DEN,DEN,2077,323,2837,7194,0.469,3377,1986,0.774,1537,...,568,2365,0.368,870,3803,1226,4829,0.519,2507,2017
CLE,CLE,1858,327,2821,6963,0.470,3275,1913,0.748,1431,...,539,2779,0.384,1067,3587,1121,4184,0.528,2208,2017
WAS,WAS,1956,336,2671,7137,0.475,3388,1812,0.784,1421,...,693,2030,0.372,756,3514,1162,5107,0.515,2632,2017
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
MIA,MIA,1991,448,2879,7218,0.450,3251,1782,0.695,1238,...,627,2658,0.349,928,3800,1208,4560,0.509,2323,2020
CHI,CHI,1796,351,2799,7205,0.453,3266,1695,0.783,1328,...,603,2123,0.351,745,3517,1159,5082,0.496,2521,2020
NYK,NYK,1646,422,2811,7241,0.433,3134,1956,0.759,1484,...,557,2421,0.340,823,3668,1151,4820,0.479,2311,2020
CLE,CLE,1698,195,2619,7184,0.444,3189,1694,0.792,1342,...,534,2388,0.355,847,3498,1106,4796,0.488,2342,2020


In [9]:
#reading in standings file to merge on stats df
try:
    standings = pd.read_excel ('standings.xlsx')
except:
    standings=pd.read_excel('standings.xlsx', engine='openpyxl')
standings.head()

,abbreviation,W,L,W/L%,PS/G,PA/G,SRS,Season
0,TOR,59,23,0.720,111.7,103.9,7.29,2017
1,BOS,55,27,0.671,104.0,100.4,3.23,2017
2,PHI,52,30,0.634,109.8,105.3,4.30,2017
3,CLE,50,32,0.610,110.9,109.9,0.59,2017
4,IND,48,34,0.585,105.6,104.2,1.18,2017


In [10]:
df_merged = pd.merge(NBA, standings, on=['abbreviation', 'Season'], how='left')
df_merged.head()

df_merged_train_data = df_merged[df_merged.Season != 2020]
df_merged_2020_data = df_merged[df_merged.Season == 2020]

In [11]:
#Testing the correlation between each team statistic and wins
corr_test = df_merged_train_data.copy()
corr_test = corr_test.drop(columns = ["abbreviation", "name", "L", "W/L%", "PS/G", "PA/G", "SRS"])

high_corr_win_cols = ["Season", "abbreviation"]

#Iterating the columns and printing mid-strength or above correlations
for col in corr_test.columns: 
    r = pearsonr(corr_test[col],corr_test["W"])
    
    if r[0] > .4 or r[0] < -0.4:
        high_corr_win_cols.append(col)
        # print(col)   
        # print(r)
        # print('-'*20)  

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/scipy/stats/stats.py:4023: PearsonRConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(PearsonRConstantInputWarning())


In [12]:
#Testing the correlation between each team statistic and rank
corr_test_rank = df_merged_train_data.copy()
corr_test_rank = corr_test_rank.drop(columns = ["abbreviation", "name", "L", "W/L%", "PS/G", "PA/G", "SRS"])

high_corr_rank_cols = ["Season", "abbreviation"]

#Iterating the columns and printing mid-strength or above correlations
for col in corr_test_rank.columns: 
    r = pearsonr(corr_test_rank[col],corr_test_rank["rank"])
    
    if r[0] > .4 or r[0] < -0.4:
        high_corr_rank_cols.append(col)
        # print(col)   
        # print(r)
        # print('-'*20)  